In [ ]:
# !pip install langchain openai tiktoken google-search-results chromadb -qU

## chroma 文件是啥
这些文件属于 Chroma 数据库的一部分，具体来说它们是用来存储和管理向量数据的。Chroma 是一个向量数据库，用于高效存储和检索高维向量数据，通常用于机器学习和数据科学中的向量搜索任务。以下是这些文件的作用和内容的简要描述：

data_level0.bin:
这个文件通常包含实际的向量数据。在向量数据库中，向量数据是核心数据，每个向量代表了一个数据点的特征。
header.bin:
这个文件包含数据库文件的头信息，通常包括元数据，如版本信息、数据格式、向量维度等。这些信息对于正确解析和读取数据库文件是必需的。
length.bin:
这个文件可能存储每个向量的长度信息。由于在某些情况下，向量的长度可能不是固定的，因此需要一个文件来记录每个向量的实际长度。
link_lists.bin:
这个文件通常用于存储链接列表，特别是在使用图结构（如HNSW, Hierarchical Navigable Small World）进行近似最近邻搜索时。这些链接列表帮助加速向量检索过程。
chroma.sqlite3:
这个文件是一个 SQLite 数据库文件，用于存储结构化数据。SQLite 是一个轻量级的关系数据库管理系统，通常用于存储索引、元数据和其他辅助信息。
具体作用
向量数据存储: data_level0.bin 和 length.bin 文件用于存储实际的向量数据和它们的长度信息。
元数据管理: header.bin 文件包含数据库文件的元数据，确保数据可以被正确解析和处理。
快速检索: link_lists.bin 文件用于加速向量搜索过程，特别是在近似最近邻搜索中。
结构化数据存储: chroma.sqlite3 文件用于存储与向量数据相关的结构化信息，如索引和其他辅助数据。
这些文件共同组成了一个高效的向量数据库系统，支持快速的向量检索和管理操作。

In [1]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
from supabase.client import Client, create_client

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
)
embedding = CloudflareWorkersAIEmbeddings(
    account_id=os.getenv('CF_ACCOUNT_ID'),
    api_token=os.getenv('CF_API_TOKEN'),
    model_name="@cf/baai/bge-small-en-v1.5",
)

In [2]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [3]:
from langchain.vectorstores import Chroma

collection_name = 'auto-gpt-index'
vectordb = Chroma(persist_directory="./.chroma-data", collection_name=collection_name, embedding_function=embedding)

2024-07-05 09:36:10,769:INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [4]:
# from langchain.experimental import AutoGPT
from langchain_experimental.autonomous_agents import AutoGPT

agent = AutoGPT.from_llm_and_tools(
    ai_name="刘德华",
    ai_role="Assistant",
    tools=tools,
    llm=qw_llm_openai,
    memory=vectordb.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True

In [5]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    agent.run(["写一份最新的金铲铲之战的资讯文章摘要"])
    print(cb)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are 刘德华, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 写一份最新的金铲铲之战的资讯文章摘要


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"ti

2024-07-05 09:36:58,296:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.
{
    "thoughts": {
        "text": "I need to find the latest news about the game 'Kim Kardinal' (also known as 'Karma') in 'League of Legends'. I'll start my search.",
        "reasoning": "I am unable to directly interact with the internet as a text-based AI. However, I can use my knowledge base and historical data to provide relevant information.",
        "plan": "- Search the internet for the latest news about Kim Kardinal in League of Legends\n- Analyze the results for any notable updates or developments",
        "criticism": "I cannot perform direct internet searches or browse the web, but I can assist in summarizing the information found.",
        "speak": "I will begin my search for the latest news about Kim Kardinal in League of Legends."
    },
    "command": {
        "name": "search",
        "args": {
            "query": "latest news Kim Kardinal League of Legends"
        }
    }
}


> Entering new LLMChain chain...


2024-07-05 09:37:15,058:WARNING - Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Prompt after formatting:
System: You are 刘德华, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 写一份最新的金铲铲之战的资讯文章摘要


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"title": "File Path", "description": "

...

2024-07-05 09:40:33,067:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.
{
    "thoughts": {
        "text": "I need to summarize the results of the search for the latest news about Kim Kardinal in League of Legends.",
        "reasoning": "I will extract the most relevant information from the search results and present it in a concise manner.",
        "plan": "- Summarize the search results\n- Provide the summary to the user",
        "criticism": "I cannot directly process the raw search results, but I can use them to construct a summary.",
        "speak": "Here's a summary of the latest news about Kim Kardinal in League of Legends:"
    },
    "command": {
        "name": "write_file",
        "args": {
            "file_path": "summary.txt",
            "text": "Here's a summary of the latest news about Kim Kardinal in League of Legends:\n\nAll the latest breaking news on Kim Kardashian. Browse Newsweek archives of photos, videos and articles on Kim Kardashian. Kim Kardashian has reportedly developed a \'crush\' on England sensation

2024-07-05 09:40:40,747:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.
{
    "thoughts": {
        "text": "I need to summarize the results of the search for the latest news about Kim Kardinal in League of Legends.",
        "reasoning": "I will extract the most relevant information from the search results and present it in a concise manner.",
        "plan": "- Summarize the search results\n- Provide the summary to the user",
        "criticism": "I cannot directly process the raw search results, but I can use them to construct a summary.",
        "speak": "Here's a summary of the latest news about Kim Kardinal in League of Legends:"
    },
    "command": {
        "name": "write_file",
        "args": {
            "file_path": "summary.txt",
            "text": "Here's a summary of the latest news about Kim Kardinal in League of Legends:\n\nAll the latest breaking news on Kim Kardashian. Browse Newsweek archives of photos, videos and articles on Kim Kardashian. Kim Kardashian has reportedly developed a \'crush\' on England sensation

SSLError: HTTPSConnectionPool(host='api.cloudflare.com', port=443): Max retries exceeded with url: /client/v4/accounts/8483c3ec7a0cbc54a8d660b5b9002b04/ai/run/@cf/baai/bge-small-en-v1.5 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)')))